In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [17]:
# Step 1: Generate dummy classification data
X, y = make_classification(n_samples=1000, n_features=20, n_classes=2, random_state=42)

# Step 2: Preprocessing
scaler = StandardScaler()
X = scaler.fit_transform(X)  # Normalize features

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

In [18]:
# Step 3: Define MLP model
def create_mlp(input_size, hidden_layers, hidden_neurons, activation_function):
    layers = []
    # Input layer to first hidden layer
    layers.append(nn.Linear(input_size, hidden_neurons))
    
    # Add hidden layers
    for _ in range(hidden_layers - 1):
        layers.append(nn.Linear(hidden_neurons, hidden_neurons))
    
    # Choose activation function
    if activation_function == 'linear':
        activation = nn.Identity()
    elif activation_function == 'Sigmoid':
        activation = nn.Sigmoid()
    elif activation_function == 'ReLU':
        activation = nn.ReLU()
    elif activation_function == 'Softmax':
        activation = nn.Softmax(dim=1)
    elif activation_function == 'Tanh':
        activation = nn.Tanh()
    
    # Output layer
    layers.append(nn.Linear(hidden_neurons, 2))  # Output 2 classes (0 or 1)
    
    # Combine all layers
    model = nn.Sequential(*layers)
    return model

In [19]:
# Step 4: Train and evaluate the model
def train_and_evaluate(model, X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor, batch_size, epochs, learning_rate):
    # DataLoader for batch training
    train_data = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

    # Optimizer (Adam) and loss function (CrossEntropyLoss for classification)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    # Training loop
    model.train()  # Set model to training mode
    for epoch in range(epochs):
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()  # Reset gradients
            output = model(X_batch)  # Forward pass
            loss = criterion(output, y_batch)  # Compute loss
            loss.backward()  # Backpropagation
            optimizer.step()  # Update model parameters

    # Evaluation after training
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():  # Disable gradient computation for efficiency
        # Training accuracy
        train_output = model(X_train_tensor)
        train_pred = torch.argmax(train_output, dim=1)  # Get predicted class
        train_accuracy = accuracy_score(y_train_tensor.numpy(), train_pred.numpy())  # Compute train accuracy
        train_loss = criterion(train_output, y_train_tensor).item()  # Compute train loss

        # Test accuracy
        test_output = model(X_test_tensor)
        test_pred = torch.argmax(test_output, dim=1)  # Get predicted class
        test_accuracy = accuracy_score(y_test_tensor.numpy(), test_pred.numpy())  # Compute test accuracy
        test_loss = criterion(test_output, y_test_tensor).item()  # Compute test loss

    return train_accuracy, train_loss, test_accuracy, test_loss  # Return results

In [ ]:
# Step 5: Experiment with hyperparameters
hidden_layers_options = [1, 2, 3]  # Number of hidden layers
hidden_neurons_options = [4, 8, 16, 32, 64]  # Number of neurons in each hidden layer
activation_functions = ['linear', 'Sigmoid', 'ReLU', 'Softmax', 'Tanh']  # Activation functions
epochs_options = [25, 50, 100, 250]  # Number of epochs
learning_rates = [0.1, 0.01, 0.001, 0.0001]  # Learning rates
batch_sizes = [ 64, 128, 256, 512]  # Batch sizes

# Tracking results
results = []  # To store the results of each experiment
best_result = None
worst_result = None
best_per_activation = {activation: None for activation in activation_functions}  # Track best result per activation function

# Loop through all combinations of hyperparameters
for hidden_layers in hidden_layers_options:
    for hidden_neurons in hidden_neurons_options:
        for activation_function in activation_functions:
            for epochs in epochs_options:
                for learning_rate in learning_rates:
                    for batch_size in batch_sizes:
                        # Print the current experiment configuration
                        print(f"Training with {hidden_layers} hidden layers, {hidden_neurons} neurons, {activation_function} activation, "
                              f"{epochs} epochs, {learning_rate} learning rate, {batch_size} batch size")

                        # Create the model for each combination of hyperparameters
                        model = create_mlp(X_train_tensor.shape[1], hidden_layers, hidden_neurons, activation_function)
                        
                        # Train and evaluate the model
                        train_acc, train_loss, test_acc, test_loss = train_and_evaluate(
                            model, X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor, batch_size, epochs, learning_rate
                        )

                        # Store the results
                        results.append({
                            'Hidden Layers': hidden_layers,
                            'Hidden Neurons': hidden_neurons,
                            'Activation Function': activation_function,
                            'Epochs': epochs,
                            'Learning Rate': learning_rate,
                            'Batch Size': batch_size,
                            'Train Accuracy': train_acc,
                            'Train Loss': train_loss,
                            'Test Accuracy': test_acc,
                            'Test Loss': test_loss
                        })

                        # Print final results for this experiment
                        print(f"Train Accuracy: {train_acc * 100:.2f}%")
                        print(f"Train Loss: {train_loss:.4f}")
                        print(f"Test Accuracy: {test_acc * 100:.2f}%")
                        print(f"Test Loss: {test_loss:.4f}\n")

                        # Update best and worst results
                        if best_result is None or test_acc > best_result['Test Accuracy']:
                            best_result = {
                                'Hyperparameters': {
                                    'Hidden Layers': hidden_layers,
                                    'Hidden Neurons': hidden_neurons,
                                    'Activation Function': activation_function,
                                    'Epochs': epochs,
                                    'Learning Rate': learning_rate,
                                    'Batch Size': batch_size
                                },
                                'Test Accuracy': test_acc
                            }
                        if worst_result is None or test_acc < worst_result['Test Accuracy']:
                            worst_result = {
                                'Hyperparameters': {
                                    'Hidden Layers': hidden_layers,
                                    'Hidden Neurons': hidden_neurons,
                                    'Activation Function': activation_function,
                                    'Epochs': epochs,
                                    'Learning Rate': learning_rate,
                                    'Batch Size': batch_size
                                },
                                'Test Accuracy': test_acc
                            }

                        # Track the best configuration for each activation function
                        if best_per_activation[activation_function] is None or test_acc > best_per_activation[activation_function]['Test Accuracy']:
                            best_per_activation[activation_function] = {
                                'Hyperparameters': {
                                    'Hidden Layers': hidden_layers,
                                    'Hidden Neurons': hidden_neurons,
                                    'Activation Function': activation_function,
                                    'Epochs': epochs,
                                    'Learning Rate': learning_rate,
                                    'Batch Size': batch_size
                                },
                                'Test Accuracy': test_acc
                            }


Training with 1 hidden layers, 4 neurons, linear activation, 25 epochs, 0.1 learning rate, 64 batch size
Train Accuracy: 88.43%
Train Loss: 0.3108
Test Accuracy: 84.67%
Test Loss: 0.3900

Training with 1 hidden layers, 4 neurons, linear activation, 25 epochs, 0.1 learning rate, 128 batch size
Train Accuracy: 88.71%
Train Loss: 0.3085
Test Accuracy: 85.00%
Test Loss: 0.3832

Training with 1 hidden layers, 4 neurons, linear activation, 25 epochs, 0.1 learning rate, 256 batch size
Train Accuracy: 88.43%
Train Loss: 0.3080
Test Accuracy: 84.33%
Test Loss: 0.3880

Training with 1 hidden layers, 4 neurons, linear activation, 25 epochs, 0.1 learning rate, 512 batch size
Train Accuracy: 89.00%
Train Loss: 0.3107
Test Accuracy: 85.00%
Test Loss: 0.3947

Training with 1 hidden layers, 4 neurons, linear activation, 25 epochs, 0.01 learning rate, 64 batch size
Train Accuracy: 89.14%
Train Loss: 0.3077
Test Accuracy: 84.67%
Test Loss: 0.3840

Training with 1 hidden layers, 4 neurons, linear activat

In [ ]:
# Display the best and worst configurations
print("\nBest Hyperparameter Configuration (Overall):")
print(f"Hidden Layers: {best_result['Hyperparameters']['Hidden Layers']}")
print(f"Hidden Neurons: {best_result['Hyperparameters']['Hidden Neurons']}")
print(f"Activation Function: {best_result['Hyperparameters']['Activation Function']}")
print(f"Epochs: {best_result['Hyperparameters']['Epochs']}")
print(f"Learning Rate: {best_result['Hyperparameters']['Learning Rate']}")
print(f"Batch Size: {best_result['Hyperparameters']['Batch Size']}")
print(f"Test Accuracy: {best_result['Test Accuracy'] * 100:.2f}%")

print("\nWorst Hyperparameter Configuration:")
print(f"Hidden Layers: {worst_result['Hyperparameters']['Hidden Layers']}")
print(f"Hidden Neurons: {worst_result['Hyperparameters']['Hidden Neurons']}")
print(f"Activation Function: {worst_result['Hyperparameters']['Activation Function']}")
print(f"Epochs: {worst_result['Hyperparameters']['Epochs']}")
print(f"Learning Rate: {worst_result['Hyperparameters']['Learning Rate']}")
print(f"Batch Size: {worst_result['Hyperparameters']['Batch Size']}")
print(f"Test Accuracy: {worst_result['Test Accuracy'] * 100:.2f}%")

# Display best configuration for each activation function
for activation in activation_functions:
    print(f"\nBest Configuration for Activation Function {activation}:")
    if best_per_activation[activation]:
        print(f"Hidden Layers: {best_per_activation[activation]['Hyperparameters']['Hidden Layers']}")
        print(f"Hidden Neurons: {best_per_activation[activation]['Hyperparameters']['Hidden Neurons']}")
        print(f"Activation Function: {activation}")
        print(f"Epochs: {best_per_activation[activation]['Hyperparameters']['Epochs']}")
        print(f"Learning Rate: {best_per_activation[activation]['Hyperparameters']['Learning Rate']}")
        print(f"Batch Size: {best_per_activation[activation]['Hyperparameters']['Batch Size']}")
        print(f"Test Accuracy: {best_per_activation[activation]['Test Accuracy'] * 100:.2f}%")
    else:
        print("No result for this activation function.")


Best Hyperparameter Configuration (Overall):
Hidden Layers: 1
Hidden Neurons: 4
Activation Function: linear
Epochs: 25
Learning Rate: 0.001
Batch Size: 32
Test Accuracy: 86.00%

Worst Hyperparameter Configuration:
Hidden Layers: 1
Hidden Neurons: 4
Activation Function: linear
Epochs: 1
Learning Rate: 10
Batch Size: 64
Test Accuracy: 30.33%

Best Configuration for Activation Function linear:
Hidden Layers: 1
Hidden Neurons: 4
Activation Function: linear
Epochs: 25
Learning Rate: 0.001
Batch Size: 32
Test Accuracy: 86.00%

Best Configuration for Activation Function Sigmoid:
No result for this activation function.

Best Configuration for Activation Function ReLU:
No result for this activation function.

Best Configuration for Activation Function Softmax:
No result for this activation function.

Best Configuration for Activation Function Tanh:
No result for this activation function.
